In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
import datetime as dt
from pyspark.sql import functions as F
from pyspark.sql import Window
import os
import re
import datetime as dt

In [3]:
# =========================
# 설정
# =========================
REGION = "경기"
SIGUNGU_CODE = "41461"  # None이면 전체
sigungu_dir = SIGUNGU_CODE if SIGUNGU_CODE else "all"

# Spark Session 설정
spark = SparkSession.builder \
    .appName(f'silver_stage_1_{REGION}_{sigungu_dir}') \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()

Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/13 10:57:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
OUT_BASE  = "/opt/spark/data/output/silver_stage_1/main"
TOJI_BASE = "/opt/spark/data/tojidaejang/parquet"

# =========================
# 1) output: 직전 분기 마지막 달 계산
# =========================
today = dt.date.today()
cur_q = (today.month - 1) // 3 + 1
prev_q = cur_q - 1
prev_q_year = today.year
if prev_q == 0:
    prev_q = 4
    prev_q_year -= 1

prev_q_last_month = prev_q * 3  # 3/6/9/12

out_path = (
    f"{OUT_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"
    f"/region={REGION}/sigungu={sigungu_dir}"
)

print("[output] path =", out_path)

# =========================
# 2) tojidaejang: 가장 최근 (year, month) 파티션 찾기
#    - 디렉토리 구조: .../year=YYYY/month=MM/region=경기
# =========================
def find_latest_year_month(base: str, region: str):
    year_re  = re.compile(r"^year=(\d{4})$")
    month_re = re.compile(r"^month=(\d{2})$")

    candidates = []
    for yname in os.listdir(base):
        ym = year_re.match(yname)
        if not ym:
            continue
        y = int(ym.group(1))
        ydir = os.path.join(base, yname)

        if not os.path.isdir(ydir):
            continue

        for mname in os.listdir(ydir):
            mm = month_re.match(mname)
            if not mm:
                continue
            m = int(mm.group(1))
            mdir = os.path.join(ydir, mname)
            if not os.path.isdir(mdir):
                continue

            # region 파티션 존재 여부 체크
            rdir = os.path.join(mdir, f"region={region}")
            if os.path.isdir(rdir):
                candidates.append((y, m))

    if not candidates:
        raise FileNotFoundError(f"no partitions found under {base} for region={region}")

    return max(candidates)  # (year, month)

latest_y, latest_m = find_latest_year_month(TOJI_BASE, REGION)

toji_daejang_path = f"{TOJI_BASE}/year={latest_y}/month={latest_m:02d}/region={REGION}"
print("[tojidaejang] latest =", f"{latest_y}-{latest_m:02d}")
print("[tojidaejang] path   =", toji_daejang_path)

# =========================
# 3) Read parquet
# =========================
silver1_df = spark.read.option("mergeSchema", "false").parquet(out_path)
toji_daejang_df    = spark.read.option("mergeSchema", "false").parquet(toji_daejang_path)

[output] path = /opt/spark/data/output/silver_stage_1/main/year=2025/month=12/region=경기/sigungu=41461
[tojidaejang] latest = 2026-01
[tojidaejang] path   = /opt/spark/data/tojidaejang/parquet/year=2026/month=01/region=경기


In [8]:
from pyspark.sql import functions as F

silver_k = (
    silver1_df
    .withColumn("dong_key", F.trim(F.col("법정동명")))
    .withColumn("bonbun_key", F.trim(F.col("본번")))
    .withColumn("date_key", F.trim(F.col("소유권변동일자")))
)

toji_k = (
    toji_daejang_df
    .withColumn("dong_key", F.trim(F.col("법정동")))
    .withColumn("bonbun_key", F.trim(F.col("본번")))
    .withColumn("date_key", F.trim(F.col("소유권변동일자")))
)

joined_df = (
    silver_k
    .join(
        toji_k.select(
            "dong_key",
            "bonbun_key",
            "date_key",
            "지주"
        ),
        on=["dong_key", "bonbun_key", "date_key"],
        how="left"
    )
)

joined_df = joined_df.drop("dong_key", "bonbun_key", "date_key")

In [44]:
silver1_df.show(1, truncate=False)

+-----------------------------+----+-------------------+----+--------------+--------------+--------+----+--------+----+------------------+--------------+------------+------+----------------------------------------+----------------+----------------+----------------+----------------+----------------+------+
|법정동명                     |본번|고유번호           |지번|소유권변동원인|소유권변동일자|토지면적|지목|공시지가|부번|관리_건축물대장_PK|옥외자주식면적|상호명      |지점명|도로명주소                              |상권업종대분류명|상권업종중분류명|상권업종소분류명|경도            |위도            |대표자|
+-----------------------------+----+-------------------+----+--------------+--------------+--------+----+--------+----+------------------+--------------+------------+------+----------------------------------------+----------------+----------------+----------------+----------------+----------------+------+
|경기도 용인시 처인구 김량장동|4   |4146110100100040002|4-2 |소유권이전    |2025-10-27    |27.7    |대  |2019000 |2   |1116120527        |0             |브레드스토리|NULL  |경기도 용인시 처인구 백옥대로1068번길 12

In [40]:
from pyspark.sql import functions as F

dup_groups_df = (
    silver1_df
    .groupBy("법정동명", "본번", "부번", "소유권변동일자")
    .count()
    .filter(F.col("count") > 1)
)

expanded_dup_df = (
    silver1_df
    .join(
        dup_groups_df.select(
            "법정동명", "본번", "부번", "소유권변동일자"
        ),
        on=["법정동명", "본번", "부번", "소유권변동일자"],
        how="inner"
    )
)


+---------------------------+----+----+--------------+-----+
|법정동명                   |본번|부번|소유권변동일자|count|
+---------------------------+----+----+--------------+-----+
|경기도 용인시 처인구 고림동|195 |1   |2010-07-15    |1    |
|경기도 용인시 처인구 고림동|195 |4   |2025-03-10    |1    |
|경기도 용인시 처인구 고림동|195 |6   |1993-05-19    |1    |
|경기도 용인시 처인구 고림동|195 |7   |2022-06-08    |1    |
|경기도 용인시 처인구 고림동|208 |13  |2025-02-28    |1    |
|경기도 용인시 처인구 고림동|208 |14  |2006-11-17    |1    |
|경기도 용인시 처인구 고림동|208 |2   |2006-11-17    |1    |
|경기도 용인시 처인구 고림동|208 |5   |2006-11-17    |1    |
|경기도 용인시 처인구 고림동|208 |6   |2006-11-17    |1    |
|경기도 용인시 처인구 고림동|208 |9   |2006-11-17    |1    |
|경기도 용인시 처인구 고림동|342 |NULL|2017-06-07    |4    |
|경기도 용인시 처인구 고림동|342 |1   |1998-04-01    |1    |
|경기도 용인시 처인구 고림동|360 |NULL|2021-01-14    |1    |
|경기도 용인시 처인구 고림동|360 |2   |2021-11-11    |15   |
|경기도 용인시 처인구 고림동|360 |3   |2016-08-09    |3    |
|경기도 용인시 처인구 고림동|370 |2   |2013-10-01    |1    |
|경기도 용인시 처인구 고림동|370 |5   |2013-10-01    |1    |

In [20]:
grouped_df = (
    joined_df
    .groupBy("법정동명", "본번", "지주")
    .agg(
        F.count("*").alias("cnt"),
        # 그룹 내 관리_건축물대장_PK가 하나라도 있으면 1 이상
        F.sum(
            F.col("관리_건축물대장_PK").isNotNull().cast("int")
        ).alias("has_building_pk")
    )
    # 전부 NULL인 그룹 제거
    .filter(F.col("has_building_pk") > 0)
    .drop("has_building_pk")
)
valid_groups_df = grouped_df.select(
    "법정동명",
    "본번",
    "지주"
)

In [21]:
expanded_df = (
    joined_df
    .join(
        valid_groups_df,
        on=["법정동명", "본번", "지주"],
        how="inner"
    )
)

In [36]:
silver_stage_1.printSchema()

NameError: name 'silver_stage_1' is not defined

In [33]:
expanded_df.select(
    "법정동명",
    "본번",
    "상호명",
    "지주",
    "소유권변동일자",
    "대표자",
    "부번",
    "지목",
    "토지면적",
    "관리_건축물대장_PK",
    "옥외자주식면적"
).orderBy(
    "법정동명",
    "본번",
    "소유권변동일자",
    "지주"
).show(55, truncate=False)

+---------------------------+----+--------------------+------+--------------+------+----+--------+--------+----------------------+--------------+
|법정동명                   |본번|상호명              |지주  |소유권변동일자|대표자|부번|지목    |토지면적|관리_건축물대장_PK    |옥외자주식면적|
+---------------------------+----+--------------------+------+--------------+------+----+--------+--------+----------------------+--------------+
|경기도 용인시 처인구 고림동|195 |흥인                |한다솜|2022-06-08    |NULL  |7   |대      |265.0   |1116130236            |0             |
|경기도 용인시 처인구 고림동|208 |정가는밥상          |김광식|2006-11-17    |조*희 |2   |대      |771.0   |11161100387371        |23            |
|경기도 용인시 처인구 고림동|208 |NULL                |김광식|2006-11-17    |NULL  |5   |전      |40.0    |NULL                  |NULL          |
|경기도 용인시 처인구 고림동|208 |NULL                |김광식|2006-11-17    |NULL  |6   |도로    |57.0    |NULL                  |NULL          |
|경기도 용인시 처인구 고림동|208 |NULL                |김광식|2006-11-17    |NULL  |9   |전      |9.0     |NULL